In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import csv
import json
from PIL import Image
import os
from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
# Method for getting data
class GetData(Dataset):
    def __init__(self, Dir, FNames, Labels, Transform):
        self.dir = Dir
        self.fnames = FNames
        self.transform = Transform
        self.lbs = Labels

    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):
        x = Image.open(os.path.join(self.dir, self.fnames[index]))
        if "train" in self.dir:
            return self.transform(x), self.lbs[index]
        elif "test" in self.dir:
            return self.transform(x), self.lbs[index]

In [ ]:
# Experiment setting
NUM_CLS = 5
BATCH = 32  # Batch size
EPOCHS = 20  # total number of epochs
LR = 1e-3  # learning rate
IM_SIZE = 256  # image size

# Device setting
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1, 2, 3, 4"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Paths for train and test images
raw_path = '/home/user/disk2/mary/xai606/raw_data/'
data_path = '/home/user/disk2/mary/xai606/final_dataset/'
train_path = f"{data_path}train_images"
test_path = f"{data_path}test_images"
model_path = '/home/user/disk2/mary/xai606/models/basemodel.pt'
label_map = 'label_num_to_disease_map.json'

In [ ]:
with open(raw_path+label_map, encoding='utf-8') as f:
    label_info = json.load(f)
print(json.dumps(label_info, indent=1))

In [ ]:
# Train and test data and corresponding Labels for train images
labels = json.load(open(f"{raw_path}{label_map}"))
train = pd.read_csv(f'{data_path}/train.csv')
test = pd.read_csv(f'{data_path}/test.csv')
X_train, y_train = train['image_id'].values, train['label'].values
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, 
                                                  random_state=2022011107, shuffle=True, stratify=y_train)
X_test, y_test = test['image_id'].values, test['label'].values
print(X_train.shape, X_val.shape, X_test.shape)


In [ ]:
# Data loading
Transform = transforms.Compose(
    [transforms.Resize((IM_SIZE, IM_SIZE)),
     transforms.RandomRotation(90),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

trainset = GetData(train_path, X_train, y_train, Transform)
trainloader = DataLoader(trainset, batch_size=BATCH, shuffle=True, num_workers=2)

valset = GetData(train_path, X_val, y_val, Transform)
valloader = DataLoader(valset, batch_size=BATCH, shuffle=True, num_workers=2)

testset = GetData(test_path, X_test, y_test, Transform)
testloader = DataLoader(testset, batch_size=BATCH, shuffle=False, num_workers=2)

In [ ]:
# Model, criterion, and optimizer
model_a = torchvision.models.resnet152()
model_a.fc = nn.Linear(8192, 5, bias=True)
model_b = model_a.cuda()
model = nn.DataParallel(model_b).to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
# Training stage
min_eval_loss = 99999
for epoch in range(EPOCHS):
    train_loss = 0.0
    train_acc = 0.0
    
    tr_loss_l = []
    tr_acc_l = []
    val_loss_l = []
    val_acc_l = []

    model = model.train()

    for i, (images, labels) in enumerate(tqdm(trainloader)):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        logits = model(images)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.detach().item()
        _, predicted = torch.max(logits, 1)
        train_acc += (predicted == labels).sum().item()
        
    print('Epoch: %d | train_loss: %.4f' % (epoch+1, train_loss / i))
    print('Epoch: %d | train_acc: %.4f' % (epoch+1, train_acc / len(trainset)))
    tr_loss_l.append(train_loss / i)
    tr_acc_l.append(train_acc / len(trainset))

    model.eval()
    
    val_loss = 0.0
    val_acc = 0.0
    for i, (images, labels) in enumerate(tqdm(valloader)):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        # att_map, logits = model(images)
        logits = model(images)
        val_loss = criterion(logits, labels)

        val_loss += val_loss.detach().item()
        val_loss = val_loss.detach().cpu().numpy()
        _, predicted = torch.max(logits, 1)
        val_acc += (predicted == labels).sum().item()
        
    print('Epoch: %d | val_loss: %.4f' % (epoch+1, val_loss / i), end="\t")
    print('Epoch: %d | val_acc: %.4f' % (epoch+1, val_acc / len(valset)))
    val_loss_l.append(val_loss / i)
    val_acc_l.append(val_acc / len(valset))
    
    if val_loss < min_eval_loss:
        print(f"Val loss{min_eval_loss} to {val_loss} upgrade, and save model {model_path}")
        min_eval_loss = val_loss
        torch.save(model, model_path)

In [ ]:
keys = ['train_acc', 'train_loss', 'val_acc', 'val_loss']
values = [tr_acc_l, tr_loss_l,  val_acc_l, val_loss_l]
train_results = {keys[i]: values[i] for i in range(len(keys))}

In [ ]:
train_results

In [ ]:
train_df = pd.DataFrame.transpose(pd.DataFrame(train_results))
train_df.to_csv (f"./basemodel_train_results.csv", index = True, header=True)

In [ ]:
basemodel = torch.load(model_path)
basemodel.eval()

test_loss = 0.0
test_acc = 0.0
test_loss_l = []
test_acc_l = []

for i, (images, labels) in enumerate(tqdm(testloader)):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # att_map, logits = model(images)
    logits = basemodel(images)
    test_loss = criterion(logits, labels)

    test_loss += test_loss.detach().item()
    _, predicted = torch.max(logits, 1)
    test_acc += (predicted == labels).sum().item()
print('Epoch: %d | Test Loss: %.4f' % (epoch+1, test_loss / i), end="\t")
print('Epoch: %d | Test Acc: %.4f' % (epoch+1, test_acc / len(testset)))
test_acc_l.append(test_acc / len(testset))
test_loss_l.append(test_loss / i)

In [ ]:
keys = ['test_acc', 'test_loss']
values = [test_acc_l, test_loss_l]
test_results = {keys[i]: values[i] for i in range(len(keys))}

In [ ]:
test_df = pd.DataFrame.transpose(pd.DataFrame(test_results))
train_df.to_csv (f"./basemodel_test_results.csv", index = True, header=True)